# Part 1 - Extracting and Saving Data from Yelp API

## Obective

- For this CodeAlong, we will be working with the Yelp API. 
- You will use the the Yelp API to search your home town for a cuisine type of your choice.
- Next class, we will then use Plotly Express to create a map with the Mapbox API to visualize the results.
    
    

## Tools You Will Use
- Part 1:
    - Yelp API:
        - Getting Started: 
            - https://www.yelp.com/developers/documentation/v3/get_started

    - `YelpAPI` python package
        -  "YelpAPI": https://github.com/gfairchild/yelpapi
- Part 2:

    - Plotly Express: https://plotly.com/python/getting-started/
        - With Mapbox API: https://www.mapbox.com/
        - `px.scatter_mapbox` [Documentation](https://plotly.com/python/scattermapbox/): 




### Applying Code From
- Efficient API Calls Lesson Link: https://login.codingdojo.com/m/376/12529/88078

In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Additional Imports
# os - for saving and loading files
# json - to work with json files
# math - to round up results
# time - to add a short pause to not overwhelm the server
import os, json, math, time

# to make yelpapi calls
from yelpapi import YelpAPI

# progress bar from tqdm_notebook
from tqdm.notebook import tqdm_notebook

In [2]:
!pip install yelpapi
!pip install tqdm

## 1. Registering for Required APIs


- Yelp: https://www.yelp.com/developers/documentation/v3/get_started


> Check the official API documentation to know what arguments we can search for: https://www.yelp.com/developers/documentation/v3/business_search

### Load Credentials and Create Yelp API Object

In [3]:
with open('/Users/james/.secret/yelp_api.json', 'r') as f:
    login = json.load(f)

In [4]:
login.keys()

dict_keys(['client-id', 'api-key'])

In [7]:
# login.items()
# login['api-key']

In [8]:
yelp = YelpAPI(login['api-key'], timeout_s=None )

### Define Search Terms and File Paths

In [10]:
# set our API call parameters and filename before the first call
location = 'Seattle, WA 98122'
term = 'pizza'

In [11]:
location.split(',')[0]

'Seattle'

In [29]:
## Specify fodler for saving data
FOLDER = 'Data/'

os.makedirs(FOLDER, exist_ok = True)
# Specifying JSON_FILE filename (can include a folder)
JSON_FILE = FOLDER+f"{location.split(',')[0]}-{term}.json"
JSON_FILE

'Data/Seattle-pizza.json'

### Check if Json File exists and Create it if it doesn't

In [30]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    
    ## If JSON_FILE included a folder:
    if len(folder) > 0:
        # create the folder
        os.makedirs(folder, exist_ok = True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    ## save the first page of results
    with open(JSON_FILE, 'w') as f:
        json.dump([],f)
        
## If it exists, inform user
else:
    print(f'[i] {JSON_FILE} already exists.')

[i] Data/Seattle-pizza.json not found. Saving empty list to file.


### Load JSON FIle and account for previous results

In [ ]:
## Load previous results and use len of results for offset

## set offset based on previous results


### Make the first API call to get the first page of data

- We will use this first result to check:
    - how many total results there are?
    - Where is the actual data we want to save?
    - how many results do we get at a time?


In [31]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp.search_query(term = term, location = location)

In [32]:
type(results)

dict

In [33]:
len(results)

3

In [34]:
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [38]:
results['total']

846

In [39]:
results['region']

{'center': {'longitude': -122.29568481445312, 'latitude': 47.61309534151991}}

In [40]:
results['businesses']

[{'id': 'EwrAoy7XbX8nfemZdDvupA',
  'alias': 'cornelly-seattle',
  'name': 'Cornelly',
  'image_url': 'https://s3-media4.fl.yelpcdn.com/bphoto/HlYYBTxylVQf6C-WZy-Ghw/o.jpg',
  'is_closed': False,
  'url': 'https://www.yelp.com/biz/cornelly-seattle?adjust_creative=xozgoVO5XbSRaEUubsjpSQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=xozgoVO5XbSRaEUubsjpSQ',
  'review_count': 105,
  'categories': [{'alias': 'pizza', 'title': 'Pizza'},
   {'alias': 'salad', 'title': 'Salad'},
   {'alias': 'pastashops', 'title': 'Pasta Shops'}],
  'rating': 4.5,
  'coordinates': {'latitude': 47.624577, 'longitude': -122.325577},
  'transactions': ['delivery', 'pickup'],
  'location': {'address1': '601 Summit Ave E',
   'address2': '',
   'address3': None,
   'city': 'Seattle',
   'zip_code': '98102',
   'country': 'US',
   'state': 'WA',
   'display_address': ['601 Summit Ave E', 'Seattle, WA 98102']},
  'phone': '',
  'display_phone': '',
  'distance': 2578.701392645466},
 {'id': 't

In [43]:
## How many results total?
df = pd.DataFrame(results['businesses'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             20 non-null     object 
 1   alias          20 non-null     object 
 2   name           20 non-null     object 
 3   image_url      20 non-null     object 
 4   is_closed      20 non-null     bool   
 5   url            20 non-null     object 
 6   review_count   20 non-null     int64  
 7   categories     20 non-null     object 
 8   rating         20 non-null     float64
 9   coordinates    20 non-null     object 
 10  transactions   20 non-null     object 
 11  location       20 non-null     object 
 12  phone          20 non-null     object 
 13  display_phone  20 non-null     object 
 14  distance       20 non-null     float64
 15  price          16 non-null     object 
dtypes: bool(1), float64(2), int64(1), object(12)
memory usage: 2.5+ KB


- Where is the actual data we want to save?

In [53]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

- Calculate how many pages of results needed to cover the total_results

In [52]:
# Import additional packages for controlling our loop
import time, math
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

43

In [54]:
for i in tqdm_notebook( range(1,n_pages+1)):
    ## The block of code we want to TRY to run
    try:
        
        time.sleep(.2)
        
        ## Read in results in progress file and check the length
        with open(JSON_FILE, 'r') as f:
            previous_results = json.load(f)
            
        ## save number of results for to use as offset
        n_results = len(previous_results)
        
        
        ## use n_results as the OFFSET 
        results = yelp_api.search_query(location=LOCATION,
                                        term=TERM, 
                                        offset=n_results+1)

        ## append new results and save to file
        previous_results.extend(results['businesses'])

        with open(JSON_FILE,'w') as f:
            json.dump(previous_results,f)
            
    ## What to do if we get an error/exception.
    except Exception as e: # saving the error message so we can print it.
        print('[!] ERROR: ',e)


  0%|          | 0/43 [00:00<?, ?it/s]

[!] ERROR:  Expecting value: line 1 column 1 (char 0)
[!] ERROR:  Expecting value: line 1 column 1 (char 0)
[!] ERROR:  Expecting value: line 1 column 1 (char 0)
[!] ERROR:  Expecting value: line 1 column 1 (char 0)
[!] ERROR:  Expecting value: line 1 column 1 (char 0)
[!] ERROR:  Expecting value: line 1 column 1 (char 0)
[!] ERROR:  Expecting value: line 1 column 1 (char 0)
[!] ERROR:  Expecting value: line 1 column 1 (char 0)
[!] ERROR:  Expecting value: line 1 column 1 (char 0)
[!] ERROR:  Expecting value: line 1 column 1 (char 0)
[!] ERROR:  Expecting value: line 1 column 1 (char 0)
[!] ERROR:  Expecting value: line 1 column 1 (char 0)
[!] ERROR:  Expecting value: line 1 column 1 (char 0)
[!] ERROR:  Expecting value: line 1 column 1 (char 0)
[!] ERROR:  Expecting value: line 1 column 1 (char 0)
[!] ERROR:  Expecting value: line 1 column 1 (char 0)
[!] ERROR:  Expecting value: line 1 column 1 (char 0)
[!] ERROR:  Expecting value: line 1 column 1 (char 0)
[!] ERROR:  Expecting value:

## Open the Final JSON File with Pandas

In [55]:
df = pd.DataFrame(JSON_FILE)

ValueError: DataFrame constructor not properly called!

In [ ]:
## convert the filename to a .csv.gz
csv_file = JSON_FILE.replace('.json','.csv.gz')
csv_file

In [ ]:
## Save it as a compressed csv (to save space)
df.to_csv(csv_file, compression='gzip', index=False)

## Bonus: compare filesize with os module's `os.path.getsize`

In [ ]:
size_json = os.path.getsize(JSON_FILE)
size_csv_gz = os.path.getsize(JSON_FILE.replace('.json','.csv.gz'))

print(f'JSON FILE: {size_json:,} Bytes')
print(f'CSV.GZ FILE: {size_csv_gz:,} Bytes')

print(f'the csv.gz is {size_json/size_csv_gz} times smaller!')

## Next Class: Processing the Results and Mapping 